# LCEL Agent核心组件
- Schema
- 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import load_tools
from langchain.agents import create_openai_functions_agent #不同的agent有不同的创建方式
from langchain.agents import AgentExecutor
#创建LLM 
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#定义agent的prompt
#https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")
#定义工具,加载预制的工具,注意有的工具需要提供LLM
tools = load_tools(["llm-math"], llm=llm)
#创建agent
agent = create_openai_functions_agent(llm, tools, prompt)
#定义agent的执行器，这里注意与老版本的不同
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "你好"})



> Entering new AgentExecutor chain...
你好！有什么可以帮助你的吗？

> Finished chain.


{'input': '你好', 'output': '你好！有什么可以帮助你的吗？'}

# Agent
- 中间步骤处理
- 提示词
- 模型配置
- 输出解释器

In [10]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain_openai import ChatOpenAI

In [19]:
#配置模型
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    )
#可用工具
@tool
def search(query: str) -> str:
    """当需要了解最新的天气信息的时候才会使用这个工具。"""
    return "晴朗,32摄氏度,无风"
tool_list = [search]
tool_list

[StructuredTool(name='search', description='search(query: str) -> str - 当需要了解最新的天气信息的时候才会使用这个工具。', args_schema=<class 'pydantic.main.searchSchema'>, func=<function search at 0x0000016078282160>)]

In [20]:
#提示词模版
# https://smith.langchain.com/hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tools'], partial_variables={'chat_history': ''}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'xml-agent-convo', 'lc_hub_commit_hash': '00f6b7470fa25a24eef6e4e3c1e44ba07189f3e91c4d987223ad232490673be8'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\nWhen you are done, respond with a final answer between

In [22]:
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log

In [23]:
#将工具列表插入到模版中
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

In [24]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

In [25]:
agent

{
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'chat_history': '', 'tools': 'search: search(query: str) -> str - 当需要了解最新的天气信息的时候才会使用这个工具。'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'xml-agent-convo', 'lc_hub_commit_hash': '00f6b7470fa25a24eef6e4e3c1e44ba07189f3e91c4d987223ad232490673be8'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</

In [26]:
#执行agent
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [27]:
agent_executor.invoke({"input": "北京今天的天气如何?"})



> Entering new AgentExecutor chain...
<tool>search</tool><tool_input>北京今天的天气晴朗,32摄氏度,无风<final_answer>北京今天的天气是晴朗,32摄氏度,无风

> Finished chain.


{'input': '北京今天的天气如何?', 'output': '北京今天的天气是晴朗,32摄氏度,无风'}

# 一个比较复杂的agent
- 工具
- 检索增强RAG
- 记忆

In [28]:
#定义工具
import os
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.utilities import SerpAPIWrapper

In [29]:
@tool
def search(query: str) -> str:
    """当需要查找实时信息的时候才会使用这个工具."""
    serp = SerpAPIWrapper()
    return serp.run(query)

print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - 当需要查找实时信息的时候才会使用这个工具.
{'query': {'title': 'Query', 'type': 'string'}}


In [30]:
search("北京今天的天气如何?")

c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"{'type': 'weather_result', 'temperature': '78', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '16%', 'wind': '11 mph', 'location': '中国北京市', 'date': 'Saturday', 'weather': 'Partly cloudy'}"

In [31]:
#RAG增强生成
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
print(f"Loaded {len(docs)} documents")
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

Loaded 1 documents


In [32]:
#搜索匹配文档块
retriever.get_relevant_documents("如何debug?")[0]

c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(page_content='The ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for this phase.Debugging‚ÄãWhen developing new LLM applications, we suggest having LangSmith tracing enabled by default.\nOftentimes, it isn‚Äôt necessary to look at every single trace. However, when things go wrong (an unexpected end result, infinite agent loop, slower than expected execution, higher than expected token usage), it‚Äôs extremely helpful to debug by looking through the application traces. LangSmith gives clear visibility and debugging information at each step of an LLM sequence, making it much easier to identify and root-cause issues.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows L

In [33]:
#把检索器加入到工具中
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！",
)
retriever_tool

Tool(name='langsmith_search', description='搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000160555816C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000160783EF140>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000016055581B20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000160783EF140>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [34]:
#可用工具集
tools = [search, retriever_tool]
tools

[StructuredTool(name='search', description='search(query: str) -> str - 当需要查找实时信息的时候才会使用这个工具.', args_schema=<class 'pydantic.main.searchSchema'>, func=<function search at 0x0000016078282520>),
 Tool(name='langsmith_search', description='搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000160555816C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000160783EF140>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000016055581B20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000160783EF140>), document_prompt=PromptTemplate(input_variables=['pa

In [40]:
#定义模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [36]:
#从hub中获取模版
from langchain import hub

# 一个最简单的模版,带记忆
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [43]:
#创建agent
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [44]:
#创建agent执行器AgentExecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [45]:
#执行
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [46]:
agent_executor.invoke({"input": "langsmith如何帮助做项目测试?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': '如何使用LangSmith进行项目测试'}`


We provide native rendering of chat messages, functions, and retrieve documents.Initial Test Set‚ÄãWhile many developers still ship an initial version of their application based on ‚Äúvibe checks‚Äù, we‚Äôve seen an increasing number of engineering teams start to adopt a more test driven approach. LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
These test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also makes it easy to run custom evaluations (both LLM and heuristic based) to score test results.Comparison View‚ÄãWhen prototyping different versions of your applications and making changes, it‚Äôs important to see whether or not you‚Äôve regressed with respect to your initial test cases.
Oftentimes, changes in the prom

{'input': 'langsmith如何帮助做项目测试?',
 'output': 'LangSmith可以帮助项目测试的方式包括：\n\n1. 提供初始测试集：LangSmith允许开发人员创建数据集，其中包含输入和参考输出的集合，然后可以使用这些数据集在LLM应用程序上运行测试。这些测试用例可以批量上传、即时创建或从应用程序跟踪中导出。LangSmith还可以轻松运行自定义评估（LLM和基于启发式的）来评分测试结果。\n\n2. 比较视图：在原型设计不同版本的应用程序并进行更改时，重要的是查看是否相对于初始测试用例出现了退化。往往，提示、检索策略或模型选择的更改可能会对应用程序产生巨大影响。\n\n3. 调试：在开发新的LLM应用程序时，建议默认启用LangSmith跟踪。当出现问题时（意外的最终结果、无限代理循环、执行速度慢于预期、令牌使用量高于预期），通过查看应用程序跟踪来调试是非常有帮助的。LangSmith在LLM序列的每个步骤都提供清晰的可见性和调试信息，使识别和排查问题变得更容易。\n\n希望这些信息能帮助您了解如何使用LangSmith进行项目测试！'}

In [49]:
#实时信息提问
agent_executor.invoke({"input": "佛山今天的天气如何?温度多少？"})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': '佛山今天的天气'}`


{'type': 'weather_result', 'temperature': '84', 'unit': 'Fahrenheit', 'precipitation': '30%', 'humidity': '85%', 'wind': '6 mph', 'location': '中国广东省佛山市', 'date': 'Saturday', 'weather': 'Isolated thunderstorms'}佛山今天的天气是孤立雷阵雨，温度为摄氏 84 度（华氏 84 度），降水概率为 30%，湿度为 85%，风速为 6 mph。

> Finished chain.


{'input': '佛山今天的天气如何?温度多少？',
 'output': '佛山今天的天气是孤立雷阵雨，温度为摄氏 84 度（华氏 84 度），降水概率为 30%，湿度为 85%，风速为 6 mph。'}

In [50]:
#前面的交互都不带记忆
agent_executor.invoke({"input": "截止目前我们都聊了什么?"})



> Entering new AgentExecutor chain...
截止目前，我们讨论了LangSmith的一些信息和工具。您有任何其他问题或需要帮助吗？

> Finished chain.


{'input': '截止目前我们都聊了什么?',
 'output': '截止目前，我们讨论了LangSmith的一些信息和工具。您有任何其他问题或需要帮助吗？'}

In [51]:
#交互时添加记忆
agent_executor.invoke({"input": "hi! 我叫bob", "chat_history": []})



> Entering new AgentExecutor chain...
你好，Bob！有什么我可以帮助你的吗？

> Finished chain.


{'input': 'hi! 我叫bob', 'chat_history': [], 'output': '你好，Bob！有什么我可以帮助你的吗？'}

In [52]:
#手动构造记忆数据
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! 我叫bob"),
            AIMessage(content="你好，Bob！很高兴认识你。有什么我可以帮助你的吗？"),
        ],
        "input": "我叫什么名字?",
    }
)



> Entering new AgentExecutor chain...
你告诉我你叫Bob。你的名字是Bob。如果你有任何其他问题或需要帮助，请随时告诉我！

> Finished chain.


{'chat_history': [HumanMessage(content='hi! 我叫bob'),
  AIMessage(content='你好，Bob！很高兴认识你。有什么我可以帮助你的吗？')],
 'input': '我叫什么名字?',
 'output': '你告诉我你叫Bob。你的名字是Bob。如果你有任何其他问题或需要帮助，请随时告诉我！'}

In [53]:
#使用RunnableWithMessageHistory自动构造记忆数据
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history = ChatMessageHistory()

In [54]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [55]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "hi! 我是tomie"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)

Parent run 41017c4a-9f6e-4b6f-8f1f-e950a23fb207 not found for run 067eaff5-c2b7-4fe8-b9b4-a15166098e35. Treating as a root run.




> Entering new AgentExecutor chain...
你好，Tomie！有什么可以帮助你的吗？

> Finished chain.


{'input': 'hi! 我是tomie', 'chat_history': [], 'output': '你好，Tomie！有什么可以帮助你的吗？'}

In [56]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)

Parent run 2a268ff0-2e2c-4802-90f5-a20a717c6075 not found for run 673a458c-3ad5-40dd-bfed-82eb5941a491. Treating as a root run.




> Entering new AgentExecutor chain...
你告诉我你叫Tomie。

> Finished chain.


{'input': '我叫什么名字?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！有什么可以帮助你的吗？')],
 'output': '你告诉我你叫Tomie。'}

In [57]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "LangSmith如何使用?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)

Parent run 057138b2-872f-45ce-8091-c6ffb2b34a4f not found for run 75d52844-f24e-424d-a872-b7e68de7ba3f. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': '如何使用LangSmith'}`


LangSmith User Guide | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for ü¶úÔ∏èüõ†Ô∏è LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, m

{'input': 'LangSmith如何使用?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！有什么可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='你告诉我你叫Tomie。')],
 'output': 'LangSmith 是一个用于LLM应用程序开发、监控和测试的平台。在这个指南中，我们将重点介绍LangSmith支持的工作流程的广度，以及它们如何适应应用程序开发生命周期的每个阶段。希望这将帮助用户最好地利用这个强大的平台，或者如果他们刚开始自己的旅程，也会给他们一些考虑的东西。\n\n- 原型设计：LLM应用程序的原型设计通常涉及在提示、模型类型、检索策略和其他参数之间进行快速实验。\n- 初始测试集：LangSmith允许开发人员创建数据集，这些数据集是输入和参考输出的集合，并使用这些数据集在LLM应用程序上运行测试。\n- 比较视图：在原型设计不同版本的应用程序并进行更改时，重要的是要看看是否相对于初始测试用例有所退化。\n- 调试：在开发新的LLM应用程序时，建议默认启用LangSmith跟踪。LangSmith在LLM序列的每个步骤提供清晰的可见性和调试信息，使识别和排除问题变得更容易。\n\n希望这些信息能帮助你更好地了解如何使用LangSmith！如果你有任何其他问题，欢迎继续问我。'}

In [58]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "截止目前我们都聊了什么?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)

Parent run 90598625-5b34-435d-8496-528844db42f7 not found for run 89e22a63-a83c-4371-a94f-9980425c3a90. Treating as a root run.




> Entering new AgentExecutor chain...
截止目前，我们聊了你的名字是Tomie，以及关于如何使用LangSmith的一些信息。如果你有任何其他问题或需要进一步的帮助，请随时告诉我！

> Finished chain.


{'input': '截止目前我们都聊了什么?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！有什么可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='你告诉我你叫Tomie。'),
  HumanMessage(content='LangSmith如何使用?'),
  AIMessage(content='LangSmith 是一个用于LLM应用程序开发、监控和测试的平台。在这个指南中，我们将重点介绍LangSmith支持的工作流程的广度，以及它们如何适应应用程序开发生命周期的每个阶段。希望这将帮助用户最好地利用这个强大的平台，或者如果他们刚开始自己的旅程，也会给他们一些考虑的东西。\n\n- 原型设计：LLM应用程序的原型设计通常涉及在提示、模型类型、检索策略和其他参数之间进行快速实验。\n- 初始测试集：LangSmith允许开发人员创建数据集，这些数据集是输入和参考输出的集合，并使用这些数据集在LLM应用程序上运行测试。\n- 比较视图：在原型设计不同版本的应用程序并进行更改时，重要的是要看看是否相对于初始测试用例有所退化。\n- 调试：在开发新的LLM应用程序时，建议默认启用LangSmith跟踪。LangSmith在LLM序列的每个步骤提供清晰的可见性和调试信息，使识别和排除问题变得更容易。\n\n希望这些信息能帮助你更好地了解如何使用LangSmith！如果你有任何其他问题，欢迎继续问我。')],
 'output': '截止目前，我们聊了你的名字是Tomie，以及关于如何使用LangSmith的一些信息。如果你有任何其他问题或需要进一步的帮助，请随时告诉我！'}